In [16]:
from pyspark.sql import HiveContext

In [17]:
sqlContext = HiveContext(sc)

In [18]:
df = spark.read.csv("/user/trab3-spark/diabetes_prediction_dataset.csv", sep =',', inferSchema = True, header = True)

In [19]:
sqlContext.sql("""create external table diabetes_patient_data (
gender string,
age integer,
hypertension int,
heart_disease int,
smoking_history string,
bmi double,
HbA1c_level double,
blood_glucose_level integer,
diabetes integer
)
row format delimited fields terminated by ','
LOCATION '/user/trab3-spark/diabetes_patient_data'""").show()

++
||
++
++



In [20]:
df.show()

+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|gender| age|hypertension|heart_disease|smoking_history|  bmi|HbA1c_level|blood_glucose_level|diabetes|
+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|Female|80.0|           0|            1|          never|25.19|        6.6|                140|       0|
|Female|54.0|           0|            0|        No Info|27.32|        6.6|                 80|       0|
|  Male|28.0|           0|            0|          never|27.32|        5.7|                158|       0|
|Female|36.0|           0|            0|        current|23.45|        5.0|                155|       0|
|  Male|76.0|           1|            1|        current|20.14|        4.8|                155|       0|
|Female|20.0|           0|            0|          never|27.32|        6.6|                 85|       0|
|Female|44.0|           0|            0|          never|19.31|  

In [21]:
df.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- smoking_history: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- HbA1c_level: double (nullable = true)
 |-- blood_glucose_level: integer (nullable = true)
 |-- diabetes: integer (nullable = true)



In [22]:
# Renomeando colunas atraeves da funcao - withColumnRenamed
df = df.withColumnRenamed("gender", "genero")
df = df.withColumnRenamed("age", "idade")
df = df.withColumnRenamed("smoking_history", "tabagismo_historia")
df = df.withColumnRenamed("hypertension", "hipertensao")
df = df.withColumnRenamed("heart_disease", "doenca_cardiaca")
df = df.withColumnRenamed("bmi", "IMC")
df = df.withColumnRenamed("HbA1c_level", "Hemoglobina_A1c")
df = df.withColumnRenamed("blood_glucose_level", "niveis_glicose")
df = df.withColumnRenamed("heart_disease", "doenca_cardiaca")

In [23]:
df.show()

+------+-----+-----------+---------------+------------------+-----+---------------+--------------+--------+
|genero|idade|hipertensao|doenca_cardiaca|tabagismo_historia|  IMC|Hemoglobina_A1c|niveis_glicose|diabetes|
+------+-----+-----------+---------------+------------------+-----+---------------+--------------+--------+
|Female| 80.0|          0|              1|             never|25.19|            6.6|           140|       0|
|Female| 54.0|          0|              0|           No Info|27.32|            6.6|            80|       0|
|  Male| 28.0|          0|              0|             never|27.32|            5.7|           158|       0|
|Female| 36.0|          0|              0|           current|23.45|            5.0|           155|       0|
|  Male| 76.0|          1|              1|           current|20.14|            4.8|           155|       0|
|Female| 20.0|          0|              0|             never|27.32|            6.6|            85|       0|
|Female| 44.0|          0|  

In [28]:
# GrupBy em gender e diabetes.
# Na media os homens que possui diabetes possui na media 60 anos e tem um IMC na meia de 31,28 - primeira linha
df.groupBy("genero","diabetes").mean("idade","IMC").show()

+------+--------+------------------+------------------+
|genero|diabetes|        avg(idade)|          avg(IMC)|
+------+--------+------------------+------------------+
|  Male|       1| 60.89477593463729| 31.28646694726477|
| Other|       0|29.555555555555557|27.379444444444445|
|Female|       0| 40.93506498308394|27.022525743654636|
|  Male|       0| 38.93420876681544|26.691107218308606|
|Female|       1| 60.99349921542255| 32.62389822909719|
+------+--------+------------------+------------------+



In [29]:
# Ordenando a variavel age
df.sort("idade").show()

+------+-----+-----------+---------------+------------------+-----+---------------+--------------+--------+
|genero|idade|hipertensao|doenca_cardiaca|tabagismo_historia|  IMC|Hemoglobina_A1c|niveis_glicose|diabetes|
+------+-----+-----------+---------------+------------------+-----+---------------+--------------+--------+
|  Male| 0.08|          0|              0|           No Info|27.32|            5.8|           155|       0|
|Female| 0.08|          0|              0|           No Info|27.32|            5.0|           155|       0|
|  Male| 0.08|          0|              0|           No Info|14.92|            3.5|            90|       0|
|Female| 0.08|          0|              0|           No Info|13.35|            3.5|           145|       0|
|Female| 0.08|          0|              0|           No Info|14.26|            6.5|           160|       0|
|Female| 0.08|          0|              0|           No Info|11.88|            5.7|            80|       0|
|  Male| 0.08|          0|  

In [30]:
# Temos no arquivo idade de 0,08 anos. SEgundo pesquisa no google crianca dessa idade pode sim ter diabetes
df.select('idade').describe().show()

+-------+-----------------+
|summary|            idade|
+-------+-----------------+
|  count|           100000|
|   mean|41.88585600000013|
| stddev|22.51683987161704|
|    min|             0.08|
|    max|             80.0|
+-------+-----------------+



In [31]:
# Pela media do nivel de glicose, exite uma diferenca considerada em quem tem diabete e quem nao tem.
df.groupBy("diabetes").mean("niveis_glicose").show(truncate=False)

+--------+-------------------+
|diabetes|avg(niveis_glicose)|
+--------+-------------------+
|1       |194.09470588235294 |
|0       |132.85246994535518 |
+--------+-------------------+



In [32]:
df.show()

+------+-----+-----------+---------------+------------------+-----+---------------+--------------+--------+
|genero|idade|hipertensao|doenca_cardiaca|tabagismo_historia|  IMC|Hemoglobina_A1c|niveis_glicose|diabetes|
+------+-----+-----------+---------------+------------------+-----+---------------+--------------+--------+
|Female| 80.0|          0|              1|             never|25.19|            6.6|           140|       0|
|Female| 54.0|          0|              0|           No Info|27.32|            6.6|            80|       0|
|  Male| 28.0|          0|              0|             never|27.32|            5.7|           158|       0|
|Female| 36.0|          0|              0|           current|23.45|            5.0|           155|       0|
|  Male| 76.0|          1|              1|           current|20.14|            4.8|           155|       0|
|Female| 20.0|          0|              0|             never|27.32|            6.6|            85|       0|
|Female| 44.0|          0|  

In [33]:
# Olhando para o valor maximo de niveis de glicose, podemos perceber que nao existe diferenca entre homens e mulheres
df.groupBy("genero").max("niveis_glicose").show()

+------+-------------------+
|genero|max(niveis_glicose)|
+------+-------------------+
|Female|                300|
| Other|                200|
|  Male|                300|
+------+-------------------+

